# Creación de la Base de Datos.

Ahora llega de momento de separar las bases de datos por tipo. 
- Absoluto
- Porcentaje



In [1]:
import pandas as pd
import os

# 
tidy_path = "..\\tidy_data\\SIM\\"
# Extraemos el nombre de los archivos.
tidy_files = os.listdir(tidy_path)
td ={file: os.path.getctime(tidy_path +"\\" + file) for file in tidy_files}

sorted_tidy = dict(sorted(td.items(), key=lambda item:item[1]))
sorted_tidy

tidy_files = list(sorted_tidy.keys())

print(tidy_files)

['ComposicionTerritorial.xlsx', 'UbicacionGeografica.xlsx', 'DinamicaPoblacional2020.xlsx', 'EstructuraSexo2020.xlsx', 'GruposVulnerables2020.xlsx', 'HechosVitales2020.xlsx', 'Salud2022.xlsx', 'Educacion2020.xlsx', 'HogaresViviendas2020.xlsx', 'MedioAmbiente2020.xlsx', 'EntornoUrbano2020.xlsx', 'Estructuras2021.xlsx', 'Automoviles2022.xlsx', 'AccidentesTransito2022.xlsx', 'SeguridadSep2023.xlsx', 'Economia2021.xlsx', 'FinanzasMunicipales2021.xlsx']


Ahora veamos un ejemplo. 

In [2]:
ex = pd.read_excel(tidy_path + "\\" + tidy_files[0])
ex.set_index(ex.columns[0], drop=True, inplace=True)
descriptor = pd.read_excel("..\\raw_data\\descriptor.xlsx")

stop = 'Aconchi'

i = 0
while ex.index[i] != stop:
    i = i + 1

ex_i = ex.iloc[i:,:]

ex_i

,Localidades,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,AGEB,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Manzanas,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18
Unnamed: 0,,,,,,,,,,,,,,,,,,
Aconchi,25,-,1,4,24,96,14,-,11,78.6,3,21.4,223,-,135,60.5,88,39.5
Agua Prieta,261,-,1,0.4,260,99.6,209,-,179,85.6,30,14.4,4115,-,3998,97.2,117,2.8
Alamos,529,-,1,0.2,528,99.8,67,-,14,20.9,53,79.1,2270,-,425,18.7,1845,81.3
Altar,351,-,1,0.3,350,99.7,68,-,31,45.6,37,54.4,394,-,341,86.5,53,13.5
Arivechi,97,-,1,1,96,99,11,-,5,45.5,6,54.5,182,-,65,35.7,117,64.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Villa Pesqueira,233,-,1,0.4,232,99.6,16,-,9,56.3,7,43.8,146,-,86,58.9,60,41.1
Yécora,478,-,1,0.2,477,99.8,30,-,9,30,21,70,431,-,172,39.9,259,60.1
Gral. Plutarco Elías Calles,202,-,1,0.5,201,99.5,82,-,54,65.9,28,34.1,1075,-,802,74.6,273,25.4


Ahora lo aplicamos a todas las bases de datos.

In [3]:

def new_dfs(files: list):
    dfs = []
    for f in files:
        ex = pd.read_excel(tidy_path + "\\" + f)
        ex.set_index(ex.columns[0], drop=True, inplace=True)
        stop = 'Aconchi'
        i = 0
        while ex.index[i] != stop:
            i = i + 1
        ex_i = ex.iloc[i:,:]
        dfs.append(ex_i)
    return dfs

data = new_dfs(tidy_files)

data[8] = data[8].iloc[:, 2:]


Ahora, vamos a leer los descriptores ya construidos. 

In [4]:
import numpy as np

sim_cols = list(descriptor['Origen'].unique())
sim_cols.remove(np.nan)
descriptor['Origen']= descriptor['Origen'].ffill()
descriptor = descriptor.set_index(['Origen','Némonico'],drop=True)
sim_cols


['Composición Territorial ',
 'Ubicación Geográfica',
 'Dinámica Poblacional',
 'Estructura Edad y Sexo',
 'Poblacion Vulnerable',
 'Hechos Vitales',
 'Salud',
 'Educación',
 'Hogares y Vivienda',
 'Medio Ambiente',
 'Entorno Urbano',
 'Estructuras',
 'Transporte',
 'Accidentes de Transito',
 'Seguridad',
 'Economia',
 'Finanzas Municipales']

Creamos una herramienta para seleccionar secciones a pesar del indice multiple. 

In [5]:
def select_level_one(level_name, df):
    return df.loc[(level_name, slice(None)), :]



Ahora, si procedemos. Primero, haremos mencion de las complicadas. 

In [6]:
sim_m = []
data_m = []
ver_f = []
for k, col in enumerate(sim_cols):
    sim_m.append(select_level_one(col, descriptor).shape[0])
    data_m.append(data[k].shape[1])
    if 2 * sim_m[k] != data_m[k]:
        ver_f.append(k)
    else:
        ver_f.append('*')
print(ver_f)

['*', 1, '*', 3, '*', '*', '*', '*', '*', '*', '*', 11, '*', '*', '*', '*', '*']


Siendo, la tabla 2, 4, y 12 las problemáticas. Las trabajaremos por aparte. Primero las generales. 

In [7]:
database_step = []
for d, df in enumerate(data):
    if ver_f[d] == d:
        database_step.append(pd.DataFrame())
        continue
    nems = select_level_one(sim_cols[d],descriptor).index.get_level_values(1)
    new_cols = []
    for nem in nems:
        new_cols += [nem + ' A', nem + ' %']

    df.columns = new_cols
    database_step.append(df)


Ahora veremos los casos particulares. 

* Ubicación Geográfica

In [8]:
nems = list(select_level_one('Ubicación Geográfica',descriptor).index.get_level_values(1))
new_nems = ['SUPTRL A','SUPTRL %']
nems = nems[:7] + new_nems + [nems[-1] + ' A']

data[1].columns = nems
data[1]

database_step[1] = data[1]

In [9]:
for v in ver_f:
    try:
        if v > 1:
            nems = select_level_one(sim_cols[v],descriptor).index.get_level_values(1)
            data[v].columns = nems
            database_step[v] = data[v]
    except:
        continue


Finalmente crearemos las bases de datos por separado. 

In [10]:
import regex as re

a = sim_cols.index('Estructuras')
Municipios = data[0].index

def xd(df, pattern):
    test_cols = list(df.columns)
    abs_variables = []
    other_variables = []
    for text in test_cols:
        if re.search(pattern, text):
            abs_variables.append( text)
        else:
            other_variables.append(text)
    return abs_variables


def xd_sep(df):
    test_cols = list(df.columns)
    other_variables = []
    for text in test_cols:
        if re.search('A$', text):
            x = 10
        elif re.search('%$', text):
            x = 10
        else:
            other_variables.append(text)
    return other_variables



test = database_step[0]
patt_abs = 'A$'
patt_per = '%$'

for d, test in enumerate(database_step):
    av= xd(test, patt_abs)
    if d == 0:
        absolute = test[av]
    if d == a:
        absolute = pd.concat([absolute, df], axis = 1)
        continue
    df = test[av]
    absolute = pd.concat([absolute, df], axis = 1)

for d, test in enumerate(database_step):
    av = xd(test, patt_per)
    if d == 0:
        percent = test[av]
    if d == 3:
        percent = pd.concat([percent, df], axis = 1)
        continue
    df = test[av]
    percent = pd.concat([percent, df], axis = 1)

for d, test in enumerate(database_step):
    av = xd_sep(test)
    if d == 0:
        other = test[av]
    if d == 3:
        continue
    if d == a:
        continue
    df = test[av]
    other = pd.concat([other, df], axis = 1)


In [13]:
percent.to_excel('..\\databases\\percent.xlsx')
absolute.to_excel('..\\databases\\absolute.xlsx')
other.to_excel('..\\databases\\other.xlsx')
